In [ ]:
from QLBM import QLBM, QLBMV2
import numpy as np
import matplotlib.pyplot as plt
import qiskit_aer
from qiskit import transpile
from qiskit_aer import AerSimulator


In [ ]:
# Domain and simulation parameters
N_POINTS_X = 256  # Number of points in the 1D grid
x_0 = np.arange(N_POINTS_X)  # Spatial grid
timesteps = 10  # Number of timesteps
c_s = 1 / np.sqrt(3)  # Speed of sound in lattice units
NUMBER_DISCRETE_VELOCITIES = 3  # For D1Q3 lattice

# Gaussian hill parameters
Psi = 0.2                # Initial concentration at the peak of the hill
Psi_ambient = 0.1        # Ambient scalar field concentration
sigma_0 = 15             # Width of the Gaussian hill
u = 0.2                  # Constant velocity field in the x-direction
x = int(N_POINTS_X / 2)  # Initial center of the Gaussian hill

# Initialize the scalar field for Quantum LBM
Psi_qlbm = np.zeros((timesteps + 1, N_POINTS_X))
Psi_qlbm[0, :] = Psi * np.exp(-(x - x_0) ** 2 / (2 * sigma_0 ** 2)) + Psi_ambient  # Initial condition

In [ ]:
simulator = AerSimulator()

In [ ]:
# Quantum LBM simulation parameters
shots = 10**7  # Number of shots for sampling
percentage_in_solution = np.zeros(timesteps)  # Track percentage of probability in solution states
density_summed = np.zeros(timesteps + 1)  # Track total density sum over time
density_summed[0] = np.sum(Psi_qlbm[0, :])  # Initial density sum

# Quantum LBM simulation loop
for t in range(timesteps):
    # Define constant velocity field
    u_LBM = np.ones(N_POINTS_X) * u

    # Create and measure all qubits in the Quantum LBM circuit
    qc_v2 = QLBMV2(density_field=Psi_qlbm[t, :], velocity_field=u_LBM, number_velocities=NUMBER_DISCRETE_VELOCITIES)
    qc_v2.measure_all()
    
    # Transpile and run the circuit
    compiled_circuit = transpile(qc_v2, simulator)
    result = simulator.run(compiled_circuit, shots=shots).result()
    counts = result.get_counts()

    # Convert counts to probabilities
    quasi_dists = {key: count / shots for key, count in counts.items()}
    
    # Calculate the number of states based on N_POINTS_X and number of velocities
    num_states = 2**(int(np.log2(3 * N_POINTS_X) + 2))
    probabilities = np.zeros(num_states)

    # Map each binary state to its probability from counts
    for state_index in range(num_states):
        binary_state = format(state_index, f'0{int(np.log2(num_states))}b')
        probabilities[state_index] = quasi_dists.get(binary_state, 0.0)

    # HYBRID APPROACH (V2): Compute the next timestep's density field in Psi_qlbm
    Psi_qlbm[t + 1, :] = (
        np.sqrt(probabilities[:N_POINTS_X]) +
        np.sqrt(probabilities[N_POINTS_X:2 * N_POINTS_X]) +
        np.sqrt(probabilities[2 * N_POINTS_X:3 * N_POINTS_X])
    ) * np.linalg.norm(Psi_qlbm[t, :].flatten())

    # Track percentage of total probability in solution states
    percentage_in_solution[t] = np.sum(probabilities[:3 * N_POINTS_X])

    # Update total density sum for the next timestep
    density_summed[t + 1] = np.sum(Psi_qlbm[t, :])

In [ ]:
# Lattice parameters for D1Q3
LATTICE_WEIGHTS = np.array([2/3, 1/6, 1/6])
LATTICE_VELOCITIES = np.array([0, 1, -1])  # Velocities for f0, f1, f2

# Initialize the classical LBM scalar field
Psi_lbm = np.zeros((timesteps + 1, N_POINTS_X))
Psi_lbm[0, :] = Psi * np.exp(-(x - x_0) ** 2 / (2 * sigma_0 ** 2)) + Psi_ambient  # Initial condition

# Classical LBM simulation loop
for t in range(timesteps):
    # Compute equilibrium distributions
    f = np.zeros((3, N_POINTS_X))
    f[0] = LATTICE_WEIGHTS[0] * Psi_lbm[t, :]  # Central distribution
    f[1] = LATTICE_WEIGHTS[1] * Psi_lbm[t, :] * (1 + (LATTICE_VELOCITIES[1] * u) / (c_s**2))  # Right-moving distribution
    f[2] = LATTICE_WEIGHTS[2] * Psi_lbm[t, :] * (1 + (LATTICE_VELOCITIES[2] * u) / (c_s**2))  # Left-moving distribution

    # Streaming step: Shift each distribution according to its lattice velocity
    f[1] = np.roll(f[1], LATTICE_VELOCITIES[1], axis=0)  # Shift f1 to the right
    f[2] = np.roll(f[2], LATTICE_VELOCITIES[2], axis=0)  # Shift f2 to the left

    # Update scalar field by summing all distributions
    Psi_lbm[t + 1, :] = np.sum(f, axis=0)

In [ ]:
# Initialize MSE, RMSE, and MAPE arrays
MSE = np.zeros(timesteps)
RMSE = np.zeros(timesteps)
MAPE = np.zeros(timesteps)

# Calculate MSE, RMSE, and MAPE for each timestep
for t in range(timesteps):
    # Mean Squared Error
    MSE[t] = np.square(np.subtract(Psi_lbm[t, :], Psi_qlbm[t, :])).mean()
    
    # Root Mean Squared Error
    RMSE[t] = np.sqrt(MSE[t])
    
    # Mean Absolute Percentage Error
    MAPE[t] = (np.abs(np.subtract(Psi_lbm[t, :], Psi_qlbm[t, :])) / Psi_lbm[t, :]).mean() * 100

In [ ]:
time_y = np.arange(0,timesteps+1,1)

plt.rcParams.update({'font.size': 22})
plt.rcParams['text.usetex'] = True
plt.figure(1,figsize=(10, 6), dpi=300)
plt.plot(time_y,density_summed/density_summed[0],'--',color='green')
plt.grid()
plt.ylabel('normalized mass',fontsize=18)
plt.xlabel('time step',fontsize=18)
plt.ticklabel_format(axis='y', style='plain')
plt.draw()


In [ ]:
plt.rcParams.update({'font.size': 22})
plt.rcParams['text.usetex'] = True
plt.figure(1,figsize=(10, 6), dpi=300)
plt.plot(time_y[1:],MAPE,'--',color='purple')
plt.grid()
plt.ylabel('MAPE in $\%$',fontsize=18)
plt.xlabel('time step',fontsize=18)
plt.draw()

In [ ]:
time = 5
plt.rcParams.update({'font.size': 22})
plt.rcParams['text.usetex'] = True
plt.figure(1,figsize=(10, 6), dpi=300)
plt.plot(x_0,Psi_qlbm[time-1,:],'-',color='orange')
plt.plot(x_0,Psi_lbm[time-1,:],'--',color='green')
plt.grid()
plt.ylabel('$\Phi$',fontsize=18)
plt.xlabel('x',fontsize=18)
plt.xlim([0,256])
plt.draw()